# Advent of Code 2020

## Introduction - Mini Essay

I chose to write solutions in Python because:

  1. Python is one of the most popular language after Javascript, so most people will immediately understand the code.
  2. For those who don't know Python, Python is generally recognized as being one of the most grokable language.
  3. Python tend to produce short code and has a lot of text parsing functions built-in.

Ironically, when I wrote these solutions, I wilfully aimed at succintness instead of clarity.
I did not go completely overboard on it though, sometimes writing helper functions where I could
have easily added more level of inlined lambdas.

... but I did use a lot of lambdas, map-function and filter-functions. This kind of code is cute
but I think that embedding multiple level of lambda, map and filter obscures the goal of the code.
Even a single map-function is more obtuse than the equivalent for-loop. The reason is simple:
a map-function has its salient relevant code in the middle of gobbledydock, while a for loop puts
the salient code in its own separate line.

Compare these two equivalent forms:

   ```
   data = list(...)
   foo = map(lambda x: salient-core-logic, data)
   ```
   
   ```
   data = list(...)
   result = list()
   for d in data:
      salient-core-logic
   ```
   
In the first case, the eye must scan a complex line to find the words salient-core-logic. In the loop form,
it is isolated on its own line, at the of a block of code. In both forms, over time, the brain rapidly
learns to ignore the irrelevant parts. But in the function-style, the core is always embedded in the middle.
The for-loop style always isolates the salient parts. I think this generalizes to most constructs of functional
languages vs imperative languages. I think it's a major reason why people have negative reactions to functional
languages. They tend to put the salient bits "in the middle", instead of "in the end".

I think this explains the resistance to adopting functional-style code in almost all languages. They all have
taken the style from the original truly functional languages without questioning the form. At the origin, the
form was dictated by the syntax of the functional languages.

For Python, imagine that map, filter, etc were built-in syntax. Then the object being operated on could be given
first and the salient-code on its own. For example, it could look like:

   ```
   data = list(...)
   result = map data as x:
      salient-core-logic
   ```
 
 The interesting thing to note is that Python list comprehension made the same mistake of adopting a syntax that
 force putting the salient-core-logic in the middle of line noise. They could have put the programmer-provided
 code in the end, but they chose to put it in the middle. Look at LISP popularity. LISP had it wrong. Readable
 code, which is governed by the core syntax of a language is important. Python imperative-style bits got it right.
 I think readability was Python not-so-hidden super-power. Unfortunately, recent Python additions have got
 it all wrong.
 


## Day 1

First day is pretty trivial: find two or three number with a product equal to a given target. Finding a triplet involves a quadratic search, but the amount of data is small enough not to be problem. I chose to use a set for rapid lookup of numbers, but I doubt it would be a problem with a simple list. The second part uses the code of the first part to simplify the problem.

It shows the style I've chosen: maps, filters and lambdas are used to parse the input.

In [1]:
def input():
    return set(map(int, clean_lines('day_01/input.txt')))

def clean_lines(file_name):
    return filter(None, map(lambda l2: l2.strip(), open(file_name)))

def match_target(values, target):
    for v in values:
        complement = target - v
        if complement in values:
            return v * complement
    return None

def part_1(values):
    return match_target(values, 2020)

def part_2(values):
    for v in values:
        matches = match_target(values, 2020 - v)
        if matches:
            return v * matches

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


197451
138233720


## Day 2

This puzzle involves validating passwords to verify if they match a set of rules encoded with numbers and letters. PArt one is about how manyu times a letter appears, part two is want to check that a letter appears at the given positions. Very trivial. The parsing of the input is more complex than the problem itself.

In [2]:
def input():
    return open('day_02/input.txt')

def password_is_valid_part_1(min, max, letter, password):
    count = sum(map(lambda x: 1 if x == letter else 0, password))
    return min <= count <= max

password_is_valid = password_is_valid_part_1

def validate_line(line):
    if not line:
        return False
    times_letter, password = line.split(':')
    password = password.strip()
    times, letter = times_letter.split()
    min, max = map(int, times.split('-'))
    return password_is_valid(min, max, letter, password)

def pos_is_valid(pos, letter, password):
    pos -= 1
    return 0 <= pos <= len(password) and password[pos] == letter

def password_is_valid_part_2(pos1, pos2, letter, password):
    return 1 == sum((pos_is_valid(pos1, letter, password), pos_is_valid(pos2, letter, password)))

def part_1(input_data):
    global password_is_valid
    password_is_valid = password_is_valid_part_1
    return sum(map(validate_line, input_data))

def part_2(input_data):
    global password_is_valid
    password_is_valid = password_is_valid_part_2
    return sum(map(validate_line, input_data))

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


434
509


## Day 3

This puzzle is about slopes in a grid filled with obstacles. It is the first day with a separate parser function for the input. For most of the following days, the input will be parsed similarly: a parser for a single line of input is used with the map function applied to each of the input lines. I suppose I could have built up a library of helper functions, but I wanted each day's solution to stand on its own, so you'll find the same pattern of code repeated each day.

Honestly, I think there must be a simpler, more functional-styled way to follow the slope, for example by generating a set of coordinates. I did not bother writing such a generator.

In [3]:
def input():
    return list(map(parse_map, map(lambda l: l.strip(), open('day_03/input.txt'))))
    
def parse_map(line: str):
    return list(map(lambda x: 1 if x == '#' else 0, line))

def count_trees(lines, right: int, down: int):
    count = 0
    x = 0
    y = 0
    for map_line in lines:
        print_line = list(map(lambda x: '.' if x == 0 else '#', map_line))
        if y % down == 0:
            if map_line[x]:
                count += 1
                print_line[x] = 'X'
            else:
                print_line[x] = 'O'
            x += right
            x %= len(map_line)
        #print(''.join(print_line))
        y += 1
    return count

def part_1(input_data):
    return count_trees(input_data, 3, 1)

def part_2(input_data):
    slopes = ((1, 1), (3, 1), (5, 1), (7, 1), (1, 2))
    counts = 1
    for right, down in slopes:
        counts *= count_trees(input_data, right, down)
    return counts

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


162
3064612320


## Day 4

Data validation... tedious even in real programs. At this point I had not yet discovered the trick to split paragraph by splitting at double-linefeed. I saw this in a collegue solution for this challenge and immediately adopted it for the following days. It really shows the difference between these toy problems and real life: in a real program, we'd assume that more than one empty line may sometimes appear, but here the input is clean.

But, this is my first time with the advent of code and I did not trust the input, so there are redundant checks when parsing the input. With teh style of the other days, the parsing could have been much shorter.

I also used functions instead of pure lambdas for some validations.

In [4]:
def input():
    return read_passports()

def clean_lines(file_name):
    return map(lambda l: l.strip(), open(file_name))

def read_passports():
    passports = []
    passport = {}
    for line in clean_lines('day_04/input.txt'):
        if not line and passport:
            passports.append(passport)
            passport = {}
            continue
        fields = line.split()
        for field in fields:
            key, value = field.split(':')
            passport[key] = value
    if passport:    
        passports.append(passport)
    return passports

def passport_has_fields(passport, required_fields, validate):
    for f, v in required_fields:
        if f not in passport:
            return False
        if validate and not v(passport[f]):
            return False
    return True

def is_height_valid(hgt):
    if len(hgt) == 4 and 'in' == hgt[2:4]:
        return 59 <= int(hgt[0:2]) <= 76
    if len(hgt) == 5 and 'cm' == hgt[3:5]:
        return 150 <= int(hgt[0:3]) <= 193
    return False

def is_hair_valid(hcr):
    return len(hcr) == 7 and hcr[0] == '#' and sum(map(lambda x: x in '0123456789abcdef', hcr[1:7])) == 6

def is_eye_valid(eye):
    return eye in {
        'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth',
    }

def is_pid_valid(pid):
    return len(pid) == 9 and sum(map(lambda x: x in '0123456789', pid)) == 9

required_fields = [
    ('byr', lambda x: len(x) == 4 and 1920 <= int(x) <= 2002 ),  # (Birth Year)
    ('iyr', lambda x: len(x) == 4 and 2010 <= int(x) <= 2020 ),  # (Issue Year)
    ('eyr', lambda x: len(x) == 4 and 2020 <= int(x) <= 2030 ),  # (Expiration Year)
    ('hgt', is_height_valid ),  # (Height)
    ('hcl', is_hair_valid ),  # (Hair Color)
    ('ecl', is_eye_valid ),  # (Eye Color)
    ('pid', is_pid_valid ),  # (Passport ID)
    # ('cid', None ),  # (Country ID)    
]

def is_passport_valid(passport):
    return passport_has_fields(passport, required_fields, False)

def is_passport_really_valid(passport):
    return passport_has_fields(passport, required_fields, True)

def part_1(input_data):
    valids = list(map(is_passport_valid, input_data))
    return sum(valids)

def part_2(input_data):
    valids = list(map(is_passport_really_valid, input_data))
    return sum(valids)

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))

245
133


## Day 5

This puzzle shows that the designers have thought about possible tricks to simplify a problem. Here seats in a plane are described by repeated subdivisions of an interval, which could suggest a binary search. But in this case the way the choices of interval are given suggest an even simpler approach: to treat the input as a binary number. They are even given in order of most-significant-bit first, making it trivial to convert to a textual binary represetation.

The real head scratcher then, is why I did not simply use the replace() function on the text instead of map and lambda?

In [5]:
def input():
    boarding_passes = filter(None, open('day_05/input.txt').read().split('\n'))
    return list(map(lambda bp: (parse_row(bp), parse_seat(bp)), boarding_passes))

def to_binary_text(s, letter_for_one):
    return map(lambda l: '1' if letter_for_one == l else '0', s)

def to_binary(s, letter_for_one):
    return int(''.join(to_binary_text(s, letter_for_one)), base=2)

def parse_row(bp):
    return to_binary(bp[0:7], 'B')

def parse_seat(bp):
    return to_binary(bp[-3:], 'R')

def part_1(row_seats):
    ids = list(map(lambda rs: rs[0] * 8 + rs[1], row_seats))
    max_id = max(ids)
    return max_id

def part_2(row_seats):
    ids = list(map(lambda rs: rs[0] * 8 + rs[1], row_seats))
    row_seats.sort()
    start_row = 1 + row_seats[0][0]
    end_row = row_seats[-1][0]
    ids = set(ids)
    for id in range(start_row * 8, end_row * 8):
        if id not in ids:
            return id

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


922
747


## Day 6

The first day I exploited the double-linefeed trick to separate the input in groups. The solution are found by a series of mapping that extract each layer of information from the previous mapping. I think it really shows off what I talked about in my opening essay: the code that does the real work of converting values from one form to another is embedded in teh middle of a line of text instead of being by itself.

In [6]:
import re

def input():
    return open('day_06/input.txt').read().split('\n\n')

letters = re.compile('''[a-z]''')
def str_to_letters_set(s):
    return set(filter(lambda x: letters.match(x), s))

def part_1(groups):
    groups_answers = map(lambda g: str_to_letters_set(g), groups)
    groups_counts = map(len, groups_answers)
    total_count = sum(groups_counts)
    return total_count

def intersect_all(s):
    result = s[0]
    for o in s[1:]:
        # Unfortunately, internal group split produces an
        # empty set for the last entry in the input, so
        # we must protect against empty sets...
        if o:
            result.intersection_update(o)
    return result

def part_2(groups):
    groups_persons = map(lambda g: g.split('\n'), groups)
    groups_persons_answers = map(lambda g: list(map(lambda p: str_to_letters_set(p), g)), groups_persons)
    groups_common_answers = map(lambda g: intersect_all(g), groups_persons_answers)
    groups_common_counts = map(len, groups_common_answers)
    total_common_count = sum(groups_common_counts)
    return total_common_count

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))

7128
3640


## Day 7

The puzzle involves building a tree of objects. In this case, which colors of bags can be in another bag of a given color. I chose to use a dict keyed by color giving the list of contained or containing colors. I also coded the tree search using a list of nodes to-be-done and nodes already seen to avoid doing work twice.

The reason I always do this instead of recursion is that in real programming, recursion can blow up the stack which is fatal to the program. Running out of memory when accumulating data is not fatal to the program, only fatal to the operation. Users are unhappy but forgiving for an operation too large to work. They are not forgiving for their program to just die and lose all their work.

Even in a programming labguage with tail recursion, having this optimization applied is dependent on writing the code is exactly the correct form. A mistake will silently work until a data set too large to work is processed.  I'll be honest and admit I've not programmed in such languages professionally. Maybe they have linter or even compiler that error out when a non-tail recursion is used?

In [7]:
import re
from collections import defaultdict

def input():
    lines = list(filter(None, open('day_07/input.txt').read().split('\n')))
    top_bags = list(map(extract_top_bag, map(lambda x: '1 ' + x, lines)))
    sub_bags = list(map(extract_sub_bags, lines))
    bags_to_sub_bags = {
        k[1]:set(v) for k, v in zip(top_bags, sub_bags)
    }
    return bags_to_sub_bags

bag_color_re = re.compile('''([0-9]+) ([a-z ]+) bag.*''')
def extract_bag_count_and_color(s):
    m = bag_color_re.match(s.strip())
    if not m:
        return (0, None)
    return (int(m.group(1)), m.group(2))

def extract_top_bag(s):
    top_bag = s.split('contain ')[0]
    return extract_bag_count_and_color(top_bag)

def extract_sub_bags(s):
    sub_bags = s.split('contain ')[1]
    return list(map(extract_bag_count_and_color, sub_bags.split(',')))

def part_1(bags_to_sub_bags):
    container_bags = defaultdict(set)
    for parent_bag, sub_bags in bags_to_sub_bags.items():
        if sub_bags:
            for count, sub_bag in sub_bags:
                container_bags[sub_bag].add(parent_bag)

    todo = set()
    seen = set()
    top_containers = set()
    todo.add('shiny gold')
    while len(todo):
        from_bag = todo.pop()
        if from_bag in seen:
            continue
        seen.add(from_bag)
        for container_bag in container_bags[from_bag]:
            todo.add(container_bag)

    seen.remove('shiny gold')
    top_shiny_gold = seen.intersection(bags_to_sub_bags.keys())
    return len(top_shiny_gold)

def part_2(bags_to_sub_bags):
    todo = list()
    todo.append((1, 'shiny gold'))
    total = 0
    while len(todo):
        count, from_bag = todo.pop()
        total += count
        if from_bag in bags_to_sub_bags:
            for sub_count, sub_bag in bags_to_sub_bags[from_bag]:
                todo.append(((count * sub_count), sub_bag))

    total -= 1
    return total

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


142
10219


## Day 8

For some reason, I decided that representing a program as an object would be nicer than all the functional style I've used previously. One of the problem withthe advent of code is that you never know what the second part will be. Code your first part too rigidly and you may be in for a rewrite. Over-engineer the first part, and you will have wasted time for no benefit.

This is in contrast to real work. Usually, YAGNI is a good idea, but at the same time, you rarely are in complete ignorance of future requirements. Still, refactor are not unusual. I normally wait until I have three different use case before trying to rewrite code more generically.

For teh puzzle in question, the first part was simply about executing the program as given. The second part required trying which operand was faulty. While there may have been a way to start from the end and work our way backward, there were no guarantee that there would be a single path backward, so it could be a lot of pain for no gain. I instead simply tried every single-operand mutations until one that worked is found.

In [8]:
def input():
    return Program(compile('day_08/input.txt'))

class Program:
    def __init__(self, ops):
        self.ops = ops
        self.acc, self.cnt = 0, 0

    @staticmethod
    def _is_already_visited(index, visited):
        return False

    def reset(self):
        self.acc, self.cnt = 0, 0

    def execute(self):
        visited = set()
        while True:
            if self.cnt in visited:
                break
            if self.cnt >= len(self.ops):
                break
            visited.add(self.cnt)
            op, value = self.ops[self.cnt]
            op(self, value)
        return self.acc


def acc_op(prg, x):
    prg.acc += x
    prg.cnt += 1

def jmp_op(prg, x):
    prg.cnt += x

def nop_op(prg, x):
    prg.cnt += 1

def compile(filename):
    ops = {
        'acc': acc_op,
        'jmp': jmp_op,
        'nop': nop_op,
    }

    def parse_op(s):
        op_and_value = s.strip().split()
        return (ops[op_and_value[0]], int(op_and_value[1]))

    return list(map(parse_op, filter(None, open(filename).read().split('\n'))))


def part_1(program):
    return program.execute()


def find_bug(prg):
    for mod in range(0,len(prg.ops)):
        mod_program = Program(prg.ops.copy())
        if mod_program.ops[mod][0] == acc_op:
            continue
        mod_op, mod_value = mod_program.ops[mod]
        mod_program.ops[mod] = (nop_op, mod_value) if mod_op == jmp_op else (jmp_op, mod_value)
        acc = mod_program.execute()
        if mod_program.cnt >= len(mod_program.ops):
            return acc
    return 0

def part_2(program):
    return find_bug(program)

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))

1586
703


## Day 9

The puzzle first part required to find a numnber that was not the sum of two previous number... that is pretty much what was done in day 1. Yet, I did not notice it and more importantly, I did not code the check as efficiently as I had done in day 1, using a quadratic search. Fortunately, the pool of numbers to search was small, so such optimization might not be beneficial given the work needed to built a set versus a linear search in an array.

The second part requires to find a range of numbers that matches a target. Again, a quadractic search of all possible start and end of range is used. I'm pretty sure a binary search approach could be done by moving one end-point not linearly, but implementing it seemed more work that it was worth.

In [9]:
def input():
    return list(map(int, filter(None, open('day_09/input.txt').read().split('\n'))))

def is_sum_of_two_in_pool(v, pool):
    for v1 in pool:
        for v2 in pool:
            if v1 == v2:
                continue
            if v == v1 + v2:
                return True
    return False

def find_not_matching_pool(pool, to_verify):
    insertion_point = 0
    for v in to_verify:
        if not is_sum_of_two_in_pool(v, pool):
            return v
        pool[insertion_point] = v
        insertion_point = (insertion_point + 1) % len(pool)
    return "All numbers match."

def part_1(input_data):
    return find_not_matching_pool(input_data[0:25], input_data[25:])

def find_matching_range(target, values):
    for start in range(0, len(values)):
        total = values[start]
        for end in range(start+1, len(values)):
            total += values[end]
            if total == target:
                return values[start:end+1]
            if total > target:
                break

def part_2(input_data):
    target = find_not_matching_pool(input_data[0:25], input_data[25:])
    interval = find_matching_range(target, input_data)
    return min(interval) + max(interval)

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))

217430975
28509180


## Day 10

This puzzle required an aha moment for the second part. As in some, but not all, puzzles, the first part was a setup for the second part instead of having an unforeseen variation in the second part. Basically, part one was a parsing and data preparation problem: converting a list of numbers into a list of the difference between successive numbers.

Part two could be extremely inneficient if one wanted to treat it like a tree search problem. The illumination was to realize that the number of way to reach a value was the sum of the ways the preceeding values could be reached. In other word, a generalization of a Fibonnacci serie.

In [10]:
def input():
    adapter_jolts = list(map(int, filter(None, open('day_10/input.txt').read().split('\n'))))
    adapter_jolts.sort()
    adapter_jolts.insert(0, 0)
    adapter_jolts.append(adapter_jolts[-1] + 3)
    return adapter_jolts

def part_1(adapter_jolts):
    adapter_diffs = list(map(lambda ab: ab[1] - ab[0], zip(adapter_jolts[0:-1], adapter_jolts[1:])))
    one_diffs = list(filter(lambda x: x == 1, adapter_diffs))
    three_diffs = list(filter(lambda x: x == 3, adapter_diffs))
    return len(one_diffs) * len(three_diffs)

def part_2(adapter_jolts):
    reach_counts = [1]
    for index in range(1, len(adapter_jolts)):
        index_reach_count = 0
        for reach in range(index -1, -1, -1):
            if adapter_jolts[index] - adapter_jolts[reach] > 3:
                break
            index_reach_count += reach_counts[reach]
        reach_counts.append(index_reach_count)

    return reach_counts[-1]

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))

2450
32396521357312


## Day 11

Cellular automata. For this version, since the input was already the whole grid and the grid was not expanding, I used a 2D array. The trick to code a cellular automata is of course to always generate an entirely new grid on each generation, to avoid modifying the data from the previous generation while it is being processed.

In [12]:
def input():
    return list(map(lambda row: [c for c in row], filter(None, open('day_11/input.txt').read().split('\n'))))

def count_neighbours(x, y, seats, max_dist):
    count = 0
    mx, my = len(seats[0]), len(seats)
    for dx, dy in ((1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)):
        for d in range(1, max_dist+1):
            tx, ty = x+dx*d, y+dy*d
            if ty < 0 or ty >= my:
                break
            if tx < 0 or tx >= mx:
                break
            c = seats[ty][tx]
            if c == '.':
                continue
            count += (c == '#')
            break
    return count

def evolve_seats(seats, max_dist, tolerance):
    new_seats = [row.copy() for row in seats]
    changed = 0
    for y in range(0, len(seats)):
        for x in range(0, len(seats[0])):
            if seats[y][x] == '.':
                continue
            elif seats[y][x] == 'L':
                if count_neighbours(x, y, seats, max_dist) == 0:
                    new_seats[y][x] = '#'
                    changed += 1
            elif seats[y][x] == '#':
                if count_neighbours(x, y, seats, max_dist) >= tolerance:
                    new_seats[y][x] = 'L'
                    changed += 1
    return new_seats, changed

def print_seats(seats):
    print('\n'.join(map(lambda row: ''.join(row), seats)) +'\n\n')

def evolve_until_stable(seats, max_dist=1, tolerance=4):
    evolution_generations = 0
    while True:
        evolution_generations += 1
        seats, changed = evolve_seats(seats, max_dist, tolerance)
        #print_seats(seats)
        if not changed:
            return seats, evolution_generations

def part_1(seats):
    final_seats, gens = evolve_until_stable(seats)
    return sum(map(lambda row: sum(map(lambda c: 1 if c == '#' else 0, row)), final_seats))

def part_2(seats):
    final_seats, gens = evolve_until_stable(seats, 10000, 5)
    return sum(map(lambda row: sum(map(lambda c: 1 if c == '#' else 0, row)), final_seats))

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))

2166
1955


## Day 12

Moving a ship around. Again, I chose to create an object. My instinct is that if something seems to have a complex persistent internal state, then an object will carry the burden of the design and simplify many common tasks. It also avoid the main pain point of using list, array or tuple: the fact that the data is anonymous. That always make the algorithm more obscure since there is no indication what is what. Having named member variable clarifies semantics tremendously.

Coding was trivial: interpret each input command as a particular movement. Each possible command is a member function. This made it easy to do part 2 where the meaning of the commands changed.

In [13]:
def input():
    return list(map(lambda s: (s[0], int(s[1:])), filter(None, open('day_12/input.txt').read().split('\n'))))

dirs = { 'E': 0, 'S': 1, 'W': 2, 'N': 3 }
rots = { 'R': 1, 'L': 3 }
moves = [(1, 0), (0, -1), (-1, 0), (0, 1)]

class ship:
    x, y, dir = (0, 0, 0)

    def turn(ship, cmd, value):
        amount = rots[cmd] * value // 90
        ship.dir = (ship.dir + amount) % 4

    def move_in_dir(ship, dir, value):
        deltas = moves[dir]
        ship.x += deltas[0] * value
        ship.y += deltas[1] * value

    def move(ship, cmd, value):
        ship.move_in_dir(dirs[cmd], value)
        
    def forward(ship, cmd, value):
        ship.move_in_dir(ship.dir, value)

    def execute_commands(ship, cmds):
        cmd_ops = {
            'E': ship.move,
            'S': ship.move,
            'W': ship.move,
            'N': ship.move,
            'L': ship.turn,
            'R': ship.turn,
            'F': ship.forward,
        }
        for c, v in cmds:
            cmd_ops[c](c, v)

def part_1(input_data):
    s = ship()
    s.execute_commands(input_data)
    return abs(s.x) + abs(s.y)

class waypoint_ship(ship):
    wx, wy = (10, 1)

    def turn(ship, cmd, value):
        amount = (rots[cmd] * value // 90) % 4
        for i in range(0, amount):
            ship.wx, ship.wy = ship.wy, -ship.wx

    def move(ship, cmd, value):
        deltas = moves[dirs[cmd]]
        ship.wx += deltas[0] * value
        ship.wy += deltas[1] * value
        
    def forward(ship, cmd, value):
        ship.x += ship.wx * value
        ship.y += ship.wy * value

def part_2(input_data):
    s = waypoint_ship()
    s.execute_commands(input_data)
    return abs(s.x) + abs(s.y)

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))

1496
63843


## Day 13

This puzzle was teh first that gave me a really hard time. I thought there would be a trick with modulo maths, but I could not see it. I filled many pages of mathematical analysis and ended-up finding the trick I needed, but it took me a lot of time. A collegue's solution made it much more obvious what was going on: that one could build up from the first bus and create a "combined bus" that had its own combined schedule.

Unfortunately, my solution was very math-based, so what is going on in the code is really not obvious.

In [14]:
def input():
    input_data = list(filter(None, open('day_13/input.txt').read().split('\n')))
    timestamp = int(input_data[0])
    buses = list(map(lambda s: 0 if 'x' == s else int(s), input_data[1].split(',')))
    return (timestamp, buses)

def part_1(t_and_b):
    timestamp, buses = t_and_b
    buses_wait_times = list(map(lambda b: b - timestamp % b if b else 1000000, buses))
    bus_and_times = list(zip(buses_wait_times, buses))
    bus_and_times.sort()
    return bus_and_times[0][0] * bus_and_times[0][1]

def part_2(t_and_b):
    timestamp, buses = t_and_b
    bus_arrival_deltas = list(range(0,len(buses)))
    bus_and_deltas = list(filter(lambda bd: bd[0], zip(buses, bus_arrival_deltas)))
    step = 1
    start = 0
    base = bus_and_deltas[0][0]
    for mod, delta in bus_and_deltas[1:]:
        new_start = first_mod_by_step(base, mod, delta, start, step)
        if new_start != start:
            start = new_start
            step *= mod
    return start * base

def first_mod_by_step(base, mod, delta, start, step):
    x = start
    for _ in range(0, max(base, mod)):
        x += step
        if (base * x + delta) % mod == 0:
            return x
    raise "bad"

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


2545
266204454441577


## Day 14

Whenever a puzzle is about emulating some hardware, I always go for an object-oriented approach. I always expect there to be a lot of internal state. In this case there was not that much internal state, so it could probably have been designed with free functions and an array.

The first part of the  puzzle was all about applying masks on bits.

The second part though added floating values that made it more complicated. The trick I used was to use a recursive function to calculate all possible combinations of floating bits. Since the number of bits was fixed, the depth of recursion was also fixed, so there is no worry about deep recursion here.

In [15]:
import re

def input():
    input_data = list(filter(None, open('day_14/input.txt').read().split('\n')))
    op_and_values = list(map(lambda v: v.split(' = '), input_data))
    return op_and_values

class Memory:
    def __init__(self):
        self.mem = {}
        self.or_mask = 0
        self.and_mask = 0b11111111111111111111111111111111111111111111111111111

    def mask_op(mem, op: str, raw_mask: str):
        mem.or_mask = int(''.join(map(lambda c: '1' if c == '1' else '0', raw_mask.strip())), 2)
        mem.and_mask = int(''.join(map(lambda c: '1' if c == 'X' else '0', raw_mask.strip())), 2)

    mem_addr_re = re.compile('''mem\[([0-9]+)\]''')
    def mem_op(mem, op: str, raw_value: str):
        addr = int(Memory.mem_addr_re.match(op).group(1))
        value = int(raw_value)
        value = (value & mem.and_mask) | mem.or_mask
        mem.mem[addr] = value

    def execute_ops(mem, ops):
        for op, raw_value in ops:
            if op.strip() == 'mask':
                mem.mask_op(op, raw_value)
            else:
                mem.mem_op(op, raw_value)

def part_1(op_and_values):
    mem = Memory()
    mem.execute_ops(op_and_values)
    return sum(mem.mem.values())

class FloatingMemory(Memory):
    def __init__(self):
        self.mem = {}
        self.or_mask = 0
        self.float_mask = 0

    def mask_op(mem, op: str, raw_mask: str):
        mem.or_mask = int(''.join(map(lambda c: '1' if c == '1' else '0', raw_mask.strip())), 2)
        mem.float_mask = int(''.join(map(lambda c: '1' if c == 'X' else '0', raw_mask.strip())), 2)

    def float_op(mem, addr, value, mask, bit):
        if bit == 36:
            mem.mem[addr] = value
            return

        or_mask = (1 << bit)
        if mask & or_mask:
            and_mask = 0b1111111111111111111111111111111111111 ^ or_mask
            mem.float_op(addr | or_mask,  value, mask, bit+1)
            mem.float_op(addr & and_mask, value, mask, bit+1)
        else:
            mem.float_op(addr, value, mask, bit+1)

    def mem_op(mem, op: str, raw_value: str):
        addr = int(Memory.mem_addr_re.match(op).group(1))
        addr = addr | mem.or_mask
        value = int(raw_value)
        mem.float_op(addr, value, mem.float_mask, 0)

def part_2(op_and_values):
    mem = FloatingMemory()
    mem.execute_ops(op_and_values)
    return sum(mem.mem.values())

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))

14862056079561
3296185383161


## Day 15

This puzzle was for some reason hard to parse in my head. The description about a game involving speaking numbers related to previously spoken numbers made me have problem pinning what was being said in the description. In my initial solution, I used more intermediary variables to clarify the rule of the game being played. The version presented here is a slightly optimized and shortened version of my initial solution, where the additional variables have been removed.

As such, it is easily the puzzle where the amount of time I spent thinking about how to solve it was the highest compared to the amount of code required.

In [16]:
def input():
    return [1,12,0,20,8,16]
    #return [0,3,6]

def play_until(spoken_when, spoken_before, just_spoken, current_turn, until_turn):
    for current_turn in range(current_turn, until_turn):
        if just_spoken in spoken_before:
            just_spoken = spoken_when[just_spoken] - spoken_before[just_spoken]
        else:
            just_spoken = 0

        if just_spoken in spoken_when:
            spoken_before[just_spoken] = spoken_when[just_spoken]
            spoken_when[just_spoken] = current_turn
        else:
            spoken_before[just_spoken] = current_turn
            spoken_when[just_spoken] = current_turn

    return just_spoken

def part_1(input_data):
    spoken_when = { k:v for v,k in enumerate(input_data) }
    return play_until(spoken_when, {}, input_data[-1], len(input_data), 2020)

def part_2(input_data):
    spoken_when = { k:v for v,k in enumerate(input_data) }
    return play_until(spoken_when, {}, input_data[-1], len(input_data), 30000000)

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


273
47205


## Day 16

Another data validation puzzle. There may be programming trick I'm not familar with to make such problem shorter, but obviously I don't know them. These puzzle always produce the long code. Given the structure of the input data, I doubt there is that much gain possible: there was multiple formats of data to be ingested.

As usual, the actual validation itself is not hard. Just apply some validation rules to each piece of data.

The second part was about using elimination to figure out where each field of data was. That is, until all field have been identified, find teh field that has only a single possible position. Yes, the ouzzle designers were kind enough to design the data such that this woudl work. They were not evil enough to make it so that each field has multiple choices and having to find the correct combination of choices that allowed all fields to be assigned, like a complicayted Sudoku. I'm grateful to not have to code a sudoku solver for a daily puzzle... I think I would have skipped it.

In [17]:
fields = None

def input():
    global fields
    input_data = list(filter(None, open('day_16/input.txt').read().split('\n\n')))
    fields = list(filter(None, input_data[0].split('\n')))
    fields = list(map(parse_field, fields))
    your_ticket = parse_ticket(input_data[1].split('\n')[1])
    nearby_tickets = list(map(parse_ticket, list(filter(None, input_data[2].split('\n')))[1:]))
    return fields, your_ticket, nearby_tickets

def parse_ticket(ticket):
    fields = filter(None, ticket.split(','))
    return list(map(int, fields))

def parse_field(field):
    name, ranges = field.split(':')
    ranges = ranges.split(' or ')
    ranges = list(map(lambda r: r.split('-'), ranges))
    ranges = list(map(lambda r: (int(r[0].strip()), int(r[1].strip())), ranges))
    return name, ranges

def valid_for_ranges(field, ranges):
    for r in ranges:
        if r[0] <= field <= r[1]:
            return True
    return False
    
def valid_field(field):
    for name, ranges in fields:
        if valid_for_ranges(field, ranges):
            return True
    return False

def extract_invalid_fields(ticket):
    invalids = []
    for field in ticket:
        if not valid_field(field):
            invalids.append(field)
    return invalids

def sum_invalid_fields(ticket):
    return sum(extract_invalid_fields(ticket))

def part_1(input_data):
    nearby_tickets = input_data[2]
    return sum(map(sum_invalid_fields, nearby_tickets))

def valid_ticket(ticket):
    return all(map(valid_field, ticket))

def nonify_invalid_ticket(ticket):
    return ticket if valid_ticket(ticket) else None

def valid_for_all_tickets(ranges, index, tickets):
    for ticket in tickets:
        if not valid_for_ranges(ticket[index], ranges):
            return False
    return True

def part_2(input_data):
    fields, your_ticket, nearby_tickets = input_data
    valid_nearby_tickets = list(filter(None, map(nonify_invalid_ticket, nearby_tickets)))
    posible_field_ids = {}
    for name, ranges in fields:
        possible_indices = set()
        for ticket_field_index in range(0, len(your_ticket)):
            if valid_for_all_tickets(ranges, ticket_field_index, valid_nearby_tickets):
                possible_indices.add(ticket_field_index)
        posible_field_ids[name] = possible_indices

    field_ids = {}
    while True:
        taken_ids = set()
        for name, possible_indices in posible_field_ids.items():
            if len(possible_indices) == 1:
                id = possible_indices.pop()
                field_ids[name] = id
                taken_ids.add(id)
        if not taken_ids:
            break
        for id in taken_ids:
            for name, possible_indices in posible_field_ids.items():
                if id in possible_indices:
                    possible_indices.remove(id)

    total = 1
    for name, id in field_ids.items():
        if name.startswith('departure'):
            total *= your_ticket[id]

    return total

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


28873
2587271823407


## Day 17

Another cellular automata... this time in 3D, and then, for part two, in 4D.

Given that this automata had no boundaries, I chose to keep the cells in a dictionary, indexed by 3D coordinates. When I discovered that part two was in 4D I decided to code it generally for any number of dimensions, to see if it could be done. It could! And I think Python is one of a few languages that can support it easily.

I was impressed by how short the final code was.

In [18]:
from itertools import product

def input():
    return list(filter(None, open('day_17/input.txt').read().split('\n')))

def prepare_cubes(input_data, dims: int):
    cubes = {}
    for y, line in enumerate(input_data):
        for x, c in enumerate(line):
            if c == '#':
                coord = (x, y) + (0,) * (dims - 2)
                cubes[coord] = 1
    return cubes

def count_neighbours(cubes, coord: tuple, dims: int):
    count = 0
    for deltas in product((-1, 0, 1), repeat=dims):
        if any(deltas):
            dc = tuple([c + d for c, d in zip(coord, deltas)])
            if dc in cubes:
                count += 1
    return count

def execute_cycle(cubes, dims: int):
    new_cubes = {}
    done = set()
    for coord in cubes:
        for deltas in product((-1, 0, 1), repeat=dims):
            dc = tuple([c + d for c, d in zip(coord, deltas)])
            if dc not in done:
                done.add(dc)
                active = dc in cubes
                count = count_neighbours(cubes, dc, dims)
                if count == 3 or (count == 2 and active):
                    new_cubes[dc] = 1
    return new_cubes

def execute_n_cycles(input_data, cycles: int, dims: int):
    cubes = prepare_cubes(input_data, dims)
    for i in range(0, cycles):
        cubes = execute_cycle(cubes, dims)
    return cubes

def part_1(input_data):
    return len(execute_n_cycles(input_data, 6, 3))

def part_2(input_data):
    return len(execute_n_cycles(input_data, 6, 4))

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


386
2276


## Day 18

A calculator, but with different order of precedenbce for additions and multiplications. I did not feel like code a syntax tree, so I coded it as a recursive parser instead. I banked on the hope that there were only a few operations to implement.

When part two changed precedence, I simply modified my initial implementation to pass a flag to delay multiplications or not. The trick was to realized I could simply accumulate all results to be multiplied and do the multiplications at the end.

I'm not proud of the solution, especially given that for a real calculator, that is definitely not the way to write it! Such is the way of these toy puzzles: not to teach proper design, but just to get things done.

In [19]:
def input():
    return list(filter(None, open('day_18/input.txt').read().split('\n')))

import re
number_re = re.compile('''[0-9]+''')

def apply_op(v1: int, v2: int, op: str, to_mult, mult_is_low: bool):
    if mult_is_low and op == '*':
        if v1:
            to_mult.append(v1)
        return v2
    else:
        if op == '+':
            return v1 + v2
        if op == '*':
            return v1 * v2
        return v2

def parse_until_closed(line: str, index: int, mult_is_low: bool):
    value = 0
    op = ''
    to_mult = []
    while index < len(line):
        number_match = number_re.match(line, index)
        if number_match:
            number = int(number_match.group(0))
            value = apply_op(value, number, op, to_mult, mult_is_low)
            index = number_match.end(0) - 1
        elif line[index] == '+' or line[index] == '*':
            op = line[index]
        elif line[index] == '(':
            number, index = parse_until_closed(line, index+1, mult_is_low)
            value = apply_op(value, number, op, to_mult, mult_is_low)
        elif line[index] == ')':
            break
        index += 1
    for v in to_mult:
        value *= v
    return value, index

def part_1(input_data):
    values = list(map(lambda l: parse_until_closed(l, 0, False)[0], input_data))
    return sum(values)

def part_2(input_data):
    values = list(map(lambda l: parse_until_closed(l, 0, True)[0], input_data))
    return sum(values)

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))

654686398176
8952864356993


## Day 19

Another puzzle where the design of the puzzle reflects the intended solution. Here, then intention was clearly to make us use regular expressions. The trick was to build a gigantic regex using the recursive description and just use Python's built-in regex engine to do all the work. The hard bit was producing the regex.

In part two, two rules are changed to be recursive... which at first seemed like a round-about way to describe the any-operator... but unfortunately, one of the recursion was recursing in the center. So instead, analyzing the changes showed that result where that both rules could be repeated any number of times, but one rule needed to be matched at least one more time than the other. So that is what I coded, explicitly.

In [20]:
def input():
    input_data = list(filter(None, open('day_19/input.txt').read().split('\n\n')))
    rules = list(filter(None, input_data[0].split('\n')))
    messages = list(filter(None, input_data[1].split('\n')))

    rules = {
        k : v for k, v in zip(
            map(lambda r: int(r.split(':')[0].strip()), rules),
            map(lambda r: list(map(lambda alt: list(map(lambda sub: sub.strip('"') if sub[0] == '"' else int(sub), alt.split())) , r.split(':')[1].strip().split('|'))), rules),
        )    
    }

    return rules, messages

def convert_rule_to_regex(rule_index, rules):
    alt_regex = []
    for alt in rules[rule_index]:
        sub_regex = []
        for sub_rule in alt:
            if type(sub_rule) is str:
                sub_regex.append(sub_rule)
            else:
                sub_regex.append(convert_rule_to_regex(sub_rule, rules))
        alt_regex.append(''.join(sub_regex))
    if len(alt_regex) == 1:
        return alt_regex[0]
    else:
        return '((' + ')|('.join(alt_regex) + '))'

import re

def part_1(input_data):
    rules, messages = input_data
    rule_0_re = convert_rule_to_regex(0, rules)
    rule_0_re = re.compile(rule_0_re)
    filtered_messages = list(filter(None, map(lambda msg: rule_0_re.fullmatch(msg), messages)))
    return len(filtered_messages)

def count_matches(msg, start, rule_re):
    count = 0
    while True:
        match = rule_re.match(msg, start)
        if not match:
            return count, start
        count += 1
        start = match.end() 
    
rule_42_re = None
rule_31_re = None

def multi_match(msg):
    start = 0
    count_42, start = count_matches(msg, start, rule_42_re)
    if count_42 < 2:
        return False
    count_31, start = count_matches(msg, start, rule_31_re)
    if count_31 < 1:
        return False
    if start != len(msg):
        return False
    return count_31 < count_42
    
def part_2(input_data):
    rules, messages = input_data

    global rule_31_re, rule_42_re
    rule_42_re = convert_rule_to_regex(42, rules)
    rule_42_re = re.compile(rule_42_re)
    rule_31_re = convert_rule_to_regex(31, rules)
    rule_31_re = re.compile(rule_31_re)

    filtered_messages = list(filter(None, map(multi_match, messages)))
    return len(filtered_messages)

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))

205
329


## Day 20

The infamous puzzle 20. Everyone I've talked to or read about moans about this puzzle. It was very tedious.

- The input format was large and complicated.
- The data were image tiles, but with unknown orientation, requiring rotations and flips to be coded.
- The matching rules required to compare borders, so yet more code.

And that was only part one. Part two required to not only place all tiles, but to cut them off to remove borders and then analyze the full content, which again could have any orientation. There were no real insight to be gained, just a lot of code to write and debug for the usual subtle errors.

In [21]:
def input():
    input_data = list(filter(None, open('day_20/input.txt').read().split('\n\n')))
    return list(map(lambda lines: Tile(lines), input_data))

class Tile:
    def __init__(self, lines):
        lines = lines.split('\n')
        self.id = int(lines[0].split()[1].strip(':'))
        self.image = [line.strip() for line in lines[1:]]
        self.build_borders()

    def build_borders(self):
        lines = self.image
        self.borders = [
            ''.join(lines[0]),
            ''.join([line[-1] for line in lines]),
            ''.join(reversed(lines[-1])),
            ''.join([line[0] for line in reversed(lines)]),
        ]
        self.flipped_borders = [
            ''.join(reversed(lines[0])),
            ''.join([line[-1] for line in reversed(lines)]),
            ''.join(lines[-1]),
            ''.join([line[0] for line in lines]),
        ]

    def flip(self):
        self.image = [''.join(reversed(line)) for line in self.image[:]]
        self.build_borders()

    def rotate(self):
        size = len(self.image)
        new_image = []
        for x in range(0, size):
            new_image.append([])
            for y in range(0, size):
                new_image[x].append(self.image[size - 1 - y][x])
        self.image = [''.join(line) for line in new_image]
        self.build_borders()


from collections import defaultdict

def count_border_ids(tiles):
    border_id_counts = defaultdict(int)

    for tile in tiles:
        for border in tile.borders:
            border_id_counts[border] += 1
        for border in tile.flipped_borders:
            border_id_counts[border] += 1
    return border_id_counts

def count_unique_borders(borders, border_id_counts):
    count = 0
    for border in borders:
        if border_id_counts[border] == 1:
            count += 1
    return count

def find_corners(tiles, border_id_counts):
    corners = set()
    corners_total = 1
    for tile in tiles:
        if count_unique_borders(tile.borders, border_id_counts) == 2 or count_unique_borders(tile.flipped_borders, border_id_counts) == 2:
            corners.add(tile)
            corners_total *= tile.id
    return corners, corners_total

def part_1(tiles):
    border_id_counts = count_border_ids(tiles)
    corners, corners_total = find_corners(tiles, border_id_counts)
    return corners_total


def find_matching_tile(tiles, searched, border_index, done):
    for tile in tiles:
        if tile.id in done:
            continue
        if searched in tile.flipped_borders:
            tile.flip()
            if searched not in tile.borders:
                raise Exception('Bad flip.')
        if searched in tile.borders:
            while searched != tile.borders[border_index]:
                tile.rotate()
            done.add(tile.id)
            return tile
    raise Exception('Image tile not found.')

def find_first_image_column(tiles, image, image_size, done, border_id_counts):
    x = 0
    for y in range(1, image_size):
        searched = image[x][y-1].flipped_borders[2]
        tile = find_matching_tile(tiles, searched, 0, done)
        if border_id_counts[tile.borders[3]] != 1:
            raise Exception('Not a proper border tile: %d.' % border_id_counts[tile.borders[1]])
        image[x].append(tile)

def find_other_image_columns(tiles, image, image_size, done):
    for x in range(1, image_size):
        image.append([])
        for y in range(0, image_size):
            searched = image[x-1][y].flipped_borders[1]
            tile = find_matching_tile(tiles, searched, 3, done)
            image[x].append(tile)

def make_full_image_tile(tiles, top_left, border_id_counts):
    while border_id_counts[top_left.borders[0]] != 1 or border_id_counts[top_left.borders[3]] != 1:
        top_left.rotate()

    done = set()
    done.add(top_left.id)

    image = [[top_left]]
    image_size = 12

    find_first_image_column(tiles, image, image_size, done, border_id_counts)
    find_other_image_columns(tiles, image, image_size, done)

    lines = ['Tile 0:']
    for y in range(0, image_size):
        for sub_y in range(1, len(image[0][0].image) - 1):
            line_parts = []
            for x in range(0, image_size):
                line_parts.append(image[x][y].image[sub_y][1:-1])
            lines.append(''.join(line_parts))
    return Tile('\n'.join(lines))

def count_monster_dots(full_image):
    monster = [
        (18, 0),
        (0, 1), (5, 1), (6, 1), (11, 1), (12, 1), (17, 1), (18, 1), (19, 1), 
        (1, 2), (4, 2), (7, 2), (10, 2), (13, 2), (16, 2), 
    ]

    size = len(full_image.image[0])
    count = 0
    for flip in range(0, 2):
        full_image.flip()
        for rot in range(0, 4):
            full_image.rotate()
            for x in range(0, size):
                for y in range(0, size):
                    for delta in monster:
                        mx = x+delta[0]
                        my = y+delta[1]
                        if mx >= size or my >= size:
                            break
                        if full_image.image[my][mx] != '#':
                            break
                    else:
                        count += 1
    return count * len(monster)

def count_image_dots(full_image):
    dot_count = 0
    size = len(full_image.image[0])
    for x in range(0, size):
        for y in range(0, size):
            if full_image.image[y][x] == '#':
                dot_count += 1
    return dot_count


def part_2(tiles):
    border_id_counts = count_border_ids(tiles)
    corners, corners_total = find_corners(tiles, border_id_counts)
    full_image = make_full_image_tile(tiles, corners.pop(), border_id_counts)
    return count_image_dots(full_image) - count_monster_dots(full_image)

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))

13224049461431
2231


## Day 21

The day of confusion. I thought, and I was not alone, that the description of the puzzle was contradictory. The puzzle is about foods, their ingredients and allergens in those ingredients. The probem was that on one hand it claimed that the list allergens in ingredient could be incomplete, but on the ther hand it claimed we could deduce the allergens associated with each ingredient... how could it be possible if any allergen could be ommited at any time? 

I guess we were supposed to assume that any allergen would be listed enough times to be isolated. Then the puzzles were mostly just set unions, intersections and differences.


In [22]:
def input():
    input_data = filter(None, open('day_21/input.txt').read().split('\n'))
    return list(map(parse_food, input_data))

def parse_food(line):
    ingredients, allergens = line.split('(contains ')
    allergens = frozenset(map(lambda a: a.strip(), allergens.strip(')').split(',')))
    ingredients = frozenset(map(lambda i: i.strip(), ingredients.split()))
    return (ingredients, allergens)

def find_potential_allergenic_ingredients(foods):
    allergens_to_ingredients = {}
    for ingredients, allergens in foods:
        for a in allergens:
            if a in allergens_to_ingredients:
                allergens_to_ingredients[a] = allergens_to_ingredients[a].intersection(ingredients)
            else:
                allergens_to_ingredients[a] = frozenset(ingredients)
    return allergens_to_ingredients

def find_all_ingredients(foods):
    all_ingredients = set()
    for ingredients, allergens in foods:
        all_ingredients.update(ingredients)
    return frozenset(all_ingredients)

def find_non_allergens(all_ingredients, allergens_to_ingredients):
    non_allergens = set(all_ingredients)
    for allergen, ingredients in allergens_to_ingredients.items():
        non_allergens.difference_update(ingredients)
    return frozenset(non_allergens)

def count_in_foods(ingredients: set, foods):
    total_count = 0
    for f in foods:
        total_count += len(ingredients.intersection(f[0]))
    return total_count

def part_1(foods):
    all_ingredients = find_all_ingredients(foods)
    allergens_to_ingredients = find_potential_allergenic_ingredients(foods)
    non_allergens = find_non_allergens(all_ingredients, allergens_to_ingredients)
    return count_in_foods(non_allergens, foods)


def find_allergens(non_allergens, allergens_to_ingredients):
    done = set(non_allergens)
    todo = set(allergens_to_ingredients.keys())
    allergens = []
    while len(todo) > 0:
        for allergen, ingredients in allergens_to_ingredients.items():
            potentials = set(ingredients.difference(done))
            if len(potentials) == 1:
                ingredient = potentials.pop()
                allergens.append((allergen, ingredient))
                done.add(ingredient)
                todo.remove(allergen)
    return allergens

def part_2(foods):
    all_ingredients = find_all_ingredients(foods)
    allergens_to_ingredients = find_potential_allergenic_ingredients(foods)
    non_allergens = find_non_allergens(all_ingredients, allergens_to_ingredients)
    allergens = find_allergens(non_allergens, allergens_to_ingredients)
    allergens.sort()

    return ','.join([ai[1] for ai in allergens])

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


2203
fqfm,kxjttzg,ldm,mnzbc,zjmdst,ndvrq,fkjmz,kjkrm


## Day 22

The puzzle was about playing a game of cards. The first part was trivial and only involved winning tricks until one player had no cards. it was so simple I even coded it to be general for N players.

Then part two got a lot more complicated. It added recursive games within games. The hard part was actually deciphering the description of the rules. Unfortunately for me, I mis-read a detail and lost a lot of time not understanding why my code was not producing the expected solution. (My error was that I thought the sub-game was played with the full remaining decks. The fact that they were not was explained in the puzzle, but only further down in the text, which I skipped for some reason...)

In [23]:
def input():
    input_data = list(filter(None, open('day_22/input.txt').read().split('\n\n')))
    return parse_decks(input_data)

def parse_deck(deck: str):
    deck = list(filter(None, deck.split('\n')))[1:]
    return list(map(int, deck))

def parse_decks(input_data):
    return list(map(parse_deck, input_data))

def score(deck: list):
    return sum([c * (len(deck) - i) for i, c in enumerate(deck)])

def play_game(decks: list):
    while all(map(len, decks)):
        takens = [d.pop(0) for d in decks]
        winner = takens.index(max(takens))
        takens.sort(reverse=True)
        decks[winner].extend(takens)
    return [score(d) for d in decks]

def part_1(decks):
    scores = play_game(decks)
    return max(scores)

def play_game_2(decks: list):
    seens = [set() for d in decks]
    while all(map(len, decks)):

        for d, s in zip(decks, seens):
            dt = tuple(d)
            if dt in s:
                return 0
            s.add(dt)

        takens = [d.pop(0) for d in decks]

        if all([len(d) >= v for d, v in zip(decks, takens)]):
            copies = [d[:v] for d, v in zip(decks, takens)]
            winner = play_game_2(copies)
            taken = takens.pop(winner)
            takens.insert(0, taken)
        else:
            winner = takens.index(max(takens))
            takens.sort(reverse=True)
        decks[winner].extend(takens)
    scores = list(map(lambda d: len(d) > 0, decks))
    return scores.index(True)

def part_2(decks):
    winner = play_game_2(decks)
    return score(decks[winner])

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))

33694
31835


## Day 23

Another of the puzzle of the advent of code 2020 that got famous... in this case due to the impossibly long running time when written using the wrong data types.

The puzzle was about cups and moving some cups around. The initial puzzle had a few cups and moved them around a hundred times. Keeping cups in an array was fast. The rules of the game were a bit confusing but not hard to code.

Then, part two increased the number of cups to a million and the number of rounds to ten millions. Suddenly, moving data around in an array was for too slow, as it involved copying a million elements for each move. I actually went down two false path at first:

- I thought maybe there was a mathematical recursion that could be analyzed to find immediately where each cup would end-up. Of course, this failed.
- I thought that maybe the design of the placement of cups would make the order of the cups repeat itself cyclicallyin a few moves (say, in the order of hundreds) and all we had to do is detect a cycle and then reduces the number of move modulo that. Unfortunately, that was not the case.

I turned out that the trick was using the correct data format. Using a linked list for the cups allowed moving a group of consecutive cups without modifying any other data. Splicing lists involves just a few operations. The other trick was to keep a dictionary indexed by cups label to find instantly the position of a given cup in the list without having to go through the entire list.

Even with these optimizations, doing ten millions round was still taking several seconds, compare to most other puzzle sub-second running time. It was so slow that I initially coded this one in C++.


In [24]:
from collections import deque

def input():
    return [9, 6, 1, 3, 8, 5, 2, 7]

# Circular list of cups.
class cup:
    def __init__(self, v):
        self.label = v
        self.next = self

cup_finder = dict

# Add a cup with the given label after the given cup,
# maintaining the circular list. Works even if the
# after-cup is None, to create the first cup.
# Add it to the cup finder, too.
def add_cup_after(after: cup, finder: cup_finder, label: int):
    new_cup = cup(label)
    if after:
        new_cup.next = after.next
        after.next = new_cup
    finder[label] = new_cup
    return new_cup

# Remove the three cups after the given one and return them.
# Note: the removed cups form a simple list, not a circular one.
def cut_next_three(current: cup):
    first = current.next
    last = current.next.next.next
    current.next = last.next
    last.next = None
    return first

# Reduce label by one, with wrap around if zero or less.
def decrease_label(label: int, max_cup: int):
    return max_cup if label == 1 else label - 1

# Verify if a label is in the linked list of cups.
# Note: linked list is not circular.
def is_label_in(picked: cup, label: int):
    while picked:
        if label == picked.label:
            return True
        picked = picked.next
    return False

# Insert the non-circular list of cups after the given cup
# which is in a circular list. Keep the resulting list circular.
def insert_after(after: cup, picked: cup):
    last = after.next
    after.next = picked
    while picked.next:
        picked = picked.next
    picked.next = last

# Execute a single crab game move.
def crab_move(current: cup, finder: cup_finder, max_label: int):
    picked_cups = cut_next_three(current)
    destination_label = decrease_label(current.label, max_label)
    while is_label_in(picked_cups, destination_label):
        destination_label = decrease_label(destination_label, max_label)
    destination_cup = finder[destination_label]
    insert_after(destination_cup, picked_cups)

def part_1(labels):
    finder = cup_finder()
    current = add_cup_after(None, finder, 4)
    last = current
    for label in labels:
        last = add_cup_after(last, finder, label)

    max_label = len(finder)

    for i in range(0, 100):
        crab_move(current, finder, max_label)
        current = current.next

    labels = []
    cup_to_print = finder[1].next
    for i in range(0, max_label-1):
        labels.append(str(cup_to_print.label))
        cup_to_print = cup_to_print.next
    return ''.join(labels)

def part_2(labels):
    finder = cup_finder()
    current = add_cup_after(None, finder, 4)
    last = current
    for label in labels:
        last = add_cup_after(last, finder, label)

    for label in range(10, 1000000 + 1):
        last = add_cup_after(last, finder, label)

    max_label = len(finder)

    for i in range(0, 10000000):
        crab_move(current, finder, max_label)
        current = current.next

    one_cup = finder[1]
    label_1 = one_cup.next.label
    label_2 = one_cup.next.next.label

    return label_1 * label_2

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


69425837
218882971435


## Day 24

Another cellular automata, but this time on a hexagonal grid. The fact that this was an cellular automata was only revealed in part two. Part one was merely a setup to have us read the input data and generate the initial state of the automata.

Like for the 3D automata, I chose to use a set of coordinates to represent the live cells. (Here, in this puzzle, the black tiles.) So the only difficulty was the hex grid. I coded hex grids inthe past, so I know their representation as a normal square grid with peculiar movement directions, so it was pretty simple to do.

So, the code ended-up pretty short and simple.

In [25]:
def input():
    input_data = list(filter(None, open('day_24/input.txt').read().split('\n')))
    tile_moves = list(map(parse_moves, input_data))
    return flip_tiles(tile_moves)

def parse_moves(line: str):
    replacements = [ ('ne', '0 '), ('se', '2 '), ('sw', '3 '), ('nw', '5 '), ('e', '1 '), ('w', '4 '), ]
    for dir, val in replacements:
        line = line.replace(dir, val)
    moves = filter(None, line.split())
    return list(map(int, moves))

dir_to_moves = ( (-1, 0), (0, -1), (1, -1), (1, 0), (0, 1), (-1, 1), )
def apply_one_move(pos: tuple, dir: int):
    return (pos[0] + dir_to_moves[dir][0], pos[1] + dir_to_moves[dir][1])

def apply_moves(grid: set, moves: list):
    pos = (0, 0)
    for m in moves:
        pos = apply_one_move(pos, m)
    if pos in grid:
        grid.remove(pos)
    else:
        grid.add(pos)
    
def flip_tiles(tile_moves: list):
    grid = set()
    for moves in tile_moves:
        apply_moves(grid, moves)
    return grid

def part_1(grid):
    return len(grid)

def count_around(grid: set, pos: tuple):
    count = 0
    for dir in range(0, 6):
        if apply_one_move(pos, dir) in grid:
            count += 1
    return count

def evolve(grid: set):
    new_grid = set()
    for pos in grid:
        count = count_around(grid, pos)
        if count == 1 or count == 2:
            new_grid.add(pos)
        for dir in range(0, 6):
            new_pos = apply_one_move(pos, dir)
            if new_pos not in grid:
                count = count_around(grid, new_pos)
                if count == 2:
                    new_grid.add(new_pos)
    return new_grid

def part_2(grid):
    for i in range(0, 100):
        grid = evolve(grid)
    return len(grid)

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


326
3979


## Day 25

This puzzle immitate public key encryption, using modulo maths. The hardest part of the puzzle was just deciphering the explanations. The code itself is trivial, just multiplications and modulos. The solution was simply to be brute-forced.

On top of that, there was not even a second part.


In [26]:
def input():
    #card_public_key = 5764801   # Example
    #door_public_key = 17807724  # Example
    card_public_key = 10943862
    door_public_key = 12721030
    return (card_public_key, door_public_key)

def transform(subject_number: int, loop_size: int, modulo: int = 20201227):
    value = 1
    for i in range(0, loop_size):
        value *= subject_number
        value %= modulo
    return value

def find_loop_size_from_public_key(subject_number: int, public_key: int, modulo: int = 20201227):
    value = 1
    for loop_size in range(1, max(public_key, modulo)):
        value *= subject_number
        value %= modulo
        if value == public_key:
            return loop_size
    raise Exception("Loop size not found.")

def part_1(input_data):
    card_public_key, door_public_key = input_data
    card_loop_size = find_loop_size_from_public_key(7, card_public_key)
    door_loop_size = find_loop_size_from_public_key(7, door_public_key)

    card_encryption_key = transform(door_public_key, card_loop_size)
    door_encryption_key = transform(card_public_key, door_loop_size)

    return door_encryption_key

def part_2(input_data):
    return None

if __name__ == '__main__':
    print(part_1(input()))
    print(part_2(input()))


5025281
None
